<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/DenseNet_121_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [2]:
#####################
# Settings
#####################

# Hyperparameters
random_seed = 1
learning_rate = 0.001
batch_size = 128
num_epochs = 20

# Architecture
num_classes = 10

# Other
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
grayscale = False

In [3]:
####################
# CIFAR-10
####################

train_indices = torch.arange(0, 48000)
valid_indices = torch.arange(48000, 50000)

train_and_valid = datasets.CIFAR10(root='data', 
                                   train=True, 
                                   transform=transforms.ToTensor(), 
                                   download=True)

train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor(), 
                                download=False)

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          num_workers=4, 
                          shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=batch_size, 
                          num_workers=4, 
                          shuffle=False)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         num_workers=4, 
                         shuffle=False)

Extracting data/cifar-10-python.tar.gz to data


In [4]:
torch.manual_seed(0)

for epoch in range(2):
    for batch_idx, (x, y) in enumerate(train_loader):
        print('Epoch: %d | Batch index: %d | Batch size: %d' % (epoch+1, batch_idx, y.size()[0]))
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 128
Epoch: 2 | Batch index: 0 | Batch size: 128


In [5]:
# Check if shuffling works properly

for images, labels in train_loader:
    pass
print(labels[:10])

for images, labels in train_loader:
    pass
print(labels[:10])

tensor([3, 0, 1, 3, 3, 5, 0, 4, 9, 4])
tensor([1, 0, 4, 1, 8, 2, 0, 3, 5, 3])


In [6]:
# Check if validation set and test set are diverse

for images, labels in valid_loader:
    pass
print(labels[:10])

for images, labels in test_loader:
    pass
print(labels[:10])

tensor([5, 0, 3, 6, 8, 7, 9, 5, 6, 6])
tensor([7, 5, 8, 0, 8, 2, 7, 0, 3, 5])


In [ ]:
############################
# Model
###########################

import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict

# Maybe hacky
def _bn_function_factory(norm, relu, conv):
    def bn_function(*inputs):
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = conv(relu(norm(concated_features)))
        return bottleneck_output
    return bn_function

class _DenseLayer(nn.Sequential):
  def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
    super(_DenseLayer, self).__init__()
    
    self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
    self.add_module('relu1', nn.ReLU(inplace=True)),
    self.add_module('conv1', nn.Conv2d(num_input_features,
                                       bn_size * growth_rate,
                                       kernel_size=1,
                                       stride=1,
                                       bias=False)),
    self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
    self.add_module('relu2', nn.ReLU(inplace=True)),
    self.add_module('conv2', nn.Conv2d(bn_size * growth_rate,
                                       growth_rate,
                                       kernel_size=3,
                                       stride=1,
                                       padding=1,
                                       bias=False))
    self.drop_rate = drop_rate
    self.memory_efficient = memory_efficient

  def forward(self, *prev_features):
    bn_function = _bn_function_factory(self.norm1, self.relu_1, self.conv1)
    pass